# BDP Multilingual Bot

---

Based on multilingual bot model, i will try to train it in QA with English and Spanish QA 

The dataset that i would use for train and test this case will be QA in English and Spanish I have combined for this model TriviaQA dataset and SQuAD_es dataset from this sites: 
 - https://github.com/ccasimiro88/TranslateAlignRetrieve
 - http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz

 

## Prepare Dataset

First I am going to create and use the information that i will use from TriviaQA dataset, selecting the params:
- Question
- Answer
- Search Results that are the context 

In [18]:
import requests
import tarfile
import os
url = "http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz"
target_path = 'triviaqa-unfiltered.tar.gz'

#Download the file if it doesn't exist
if target_path not in os.listdir():
    print("Downloading file")
    # Dowload the file 
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        print("Saving file")
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

    # Unzip the file
    with tarfile.open(target_path, "r:gz") as tar:
        tar.extractall(path=".")
else:
    print("File already exists")

File already exists


In [19]:
# create a  Train Dataframe from the json file with the variables that i need of TriviaQA
import os
import pandas as pd
import json

if 'triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-train.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'context': context,
                'question': question,
                'answers': [answer]
            })
        # Create a DataFrame from the list of dictionaries
        df_tr = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        df_tr.to_csv('triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame
    else:
        print("Json has not the dxpected format")
else:
    df_tr = pd.read_csv('triviaqa-unfiltered.csv')
    
df_tr.head()

,context,question,answers
0,"Welcome to GoComics.com, the world's largest c...",Who was President when the first Peanuts carto...,[Harry Truman]
1,The Nobel Prize in Literature 1930 Sinclair .....,Which American-born Sinclair won the Nobel Pri...,[Sinclair Lewis]
2,Dame Judi Dench is a renowned ... Born in Engl...,Where in England was Dame Judi Dench born?,[York]
3,Our expert has answerd your question. Meet our...,"William Christensen of Madison, New Jersey, ha...",[Beer Cans]
4,Third Man Records Launches First Record Played...,In which decade did Billboard magazine first p...,[30s]


In [20]:
# create a  Test Dataframe from the json file with the variables that i need of TriviaQA

if 'test-triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-dev.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'context': context,
                'question': question,
                'answers': [answer]
            })

        # Create a DataFrame from the list of dictionaries
        test_tr = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        test_tr.to_csv('test-triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame

    else:
        print("Json has not the dxpected format")
else:
    test_tr = pd.read_csv('test-triviaqa-unfiltered.csv')

test_tr.head()

,context,question,answers
0,Alice Cooper's The Man Behind the Mask Music V...,Who was the man behind The Chipmunks?,[David Seville]
1,"Jamie Lee Curtis, Actress: True Lies. Jamie Le...",What star sign is Jamie Lee Curtis?,[Scorpio]
2,"The official website for Andrew Lloyd Webber, ...",Which Lloyd Webber musical premiered in the US...,[Sunset Boulevard]
3,The history and complete text of the 1917 Balf...,Who was the next British Prime Minister after ...,[Campbell-Bannerman]
4,... credits and award information for 70 Numbe...,Who had a 70s No 1 hit with Kiss You All Over?,[Exile]


Now i am going to prepare the SQuAD-es dataset, selecting:
- Context
- Questions
- Answers 

In [21]:
# Download dataset if it doesn't exist
import os
from datasets import load_dataset

target_path = 'squad_es'

if target_path not in os.listdir():
        # Dowload the file showing the progress bar
    print("Downloading file")
    dataset = load_dataset("squad_es", 'v1.1.0')  
    dataset.save_to_disk(target_path)  
else:
    print("File already exists")

File already exists


In [22]:
from datasets import load_from_disk
import pandas as pd

# Cargar el conjunto de datos desde el disco
if "squad_train.csv" not in os.listdir() or "squad_test.csv" not in os.listdir():
    dataset = load_from_disk('squad_es')

    # Convertir el conjunto de datos en DataFrame de Pandas
    # Asumiendo que quieres convertir la parte 'train' del conjunto de datos
    df_sq = pd.DataFrame(dataset['train'])
    test_sq = pd.DataFrame(dataset['validation'])

    # eliminate the columns that i dont need
    df_sq.pop('id')
    df_sq.pop('title')
    test_sq.pop('id')
    test_sq.pop('title')  
    
    df_sq['answers'] = df_sq['answers'].apply(lambda x: x['text'])
    test_sq['answers'] = test_sq['answers'].apply(lambda x: x['text'])


    df_sq.to_csv('squad_train.csv', index=False)
    test_sq.to_csv('squad_test.csv', index=False)
    

else:
    df_sq = pd.read_csv('squad_train.csv')
    test_sq = pd.read_csv('squad_test.csv')
    


In [26]:
# Now, combine the two dataframes in one, mix the dataframes

if 'train.csv' not in os.listdir() or 'test.csv' not in os.listdir():
    df = pd.concat([df_tr, df_sq], ignore_index=True)
    test = pd.concat([test_tr, test_sq], ignore_index=True)
else:
    df = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')


,context,question,answers
0,"Welcome to GoComics.com, the world's largest c...",Who was President when the first Peanuts carto...,[Harry Truman]
1,The Nobel Prize in Literature 1930 Sinclair .....,Which American-born Sinclair won the Nobel Pri...,[Sinclair Lewis]
2,Dame Judi Dench is a renowned ... Born in Engl...,Where in England was Dame Judi Dench born?,[York]
3,Our expert has answerd your question. Meet our...,"William Christensen of Madison, New Jersey, ha...",[Beer Cans]
4,Third Man Records Launches First Record Played...,In which decade did Billboard magazine first p...,[30s]


Now, we have got both dataframes combined, lets tokenize the information

In [24]:
from transformers import BertTokenizer, BertModel
import torch

# Verify if Cuda is available to use GPU
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

# use multilingual cased model and its tokenizer
tokenizer_cased = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_cased = BertModel.from_pretrained('bert-base-multilingual-cased')


CUDA (GPU) está disponible en tu sistema.


In [28]:


if not os.path.exists('train.csv'):
    # Function to tokenize the text

    def tokenize(text):
        return tokenizer_cased.encode_plus(
            text,
            add_special_tokens=True,#add [CLS], [SEP] (special tokens) at the beginning and end of the sentence
            max_length=512, #maximum length of a sentence in tokens
            padding='max_length', #add [PAD]s at the end of sentences to reach max_length
            truncation=True, #truncates sentences to max_length if they exceed it
            return_attention_mask=True, #return attention masks to know what tokens to attend to
            return_tensors='pt' #return PyTorch tensors (also works with TensorFlow) being able to use them directly in PyTorch models with GPU
        )

    # Apply the tokenize function to the question and context columns
    # input_ids are the tokenized text (number that represent the word in the vocabulary)
    df['question'] = df['question'].apply(lambda x: tokenize(x)['input_ids']) #apply the tokenize function to the question column using the input_ids
    test['question'] = test['question'].apply(lambda x: tokenize(x)['input_ids'])
    df['context'] = df['context'].apply(lambda x: tokenize(x)['input_ids'])
    test['context'] = test['context'].apply(lambda x: tokenize(x)['input_ids'])

    # due to i want a moder able to genereate a answer, i need to tokenize the answers
    # in the other hand if i want to use a model to select an answer i dont need to tokenize the answers but i could do it anyways 
    df['answer'] = df['answer'].apply(lambda x: tokenize(x)['input_ids'])
    test['answer'] = test['answer'].apply(lambda x: tokenize(x)['input_ids'])
    
    df.to_csv('train.csv', index=False)
    test.to_csv('test.csv', index=False)

else:
    df = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
df.head()